In [9]:
#1. generate hyperpriors
import numpy as np

# Function to generate the hyperpriors with dense sampling in the lower range and larger increments in the higher range
def generate_hyperpriors(lower_bound, upper_bound, dense_increment, expansion_factor, max_value):
    # Start with dense sampling in the lower range
    hyperpriors = np.arange(lower_bound, upper_bound, dense_increment).tolist()
    
    # Continue with larger increments in the higher range
    current_value = upper_bound
    while current_value <= max_value:
        hyperpriors.append(current_value)
        increment = (current_value - hyperpriors[-2]) * expansion_factor
        # Ensure that the increment does not shrink smaller than the dense increment
        increment = max(increment, dense_increment)
        current_value += increment

    # Ensure the last value does not exceed the maximum value set
    hyperpriors = [x for x in hyperpriors if x <= max_value]
    
    return hyperpriors


In [10]:
# Parameters for the hyperpriors
lower_bound = 0  # Starting point for the dense sampling
upper_bound = 5.0  # End point for the dense sampling
dense_increment = 0.3  # Increment for the dense sampling
expansion_factor = 1.25  # Factor by which the increment increases in the expanded range
max_value = 100.0  # Maximum value for the hyperpriors

# Generate the list of hyperpriors
hyperpriors = generate_hyperpriors(lower_bound, upper_bound, dense_increment, expansion_factor, max_value)
print(len(hyperpriors))
print(hyperpriors)

37
[0.0, 0.3, 0.6, 0.8999999999999999, 1.2, 1.5, 1.7999999999999998, 2.1, 2.4, 2.6999999999999997, 3.0, 3.3, 3.5999999999999996, 3.9, 4.2, 4.5, 4.8, 5.0, 5.3, 5.675, 6.14375, 6.7296875, 7.462109375, 8.37763671875, 9.522045898437502, 10.95255737304688, 12.7406967163086, 14.97587089538575, 17.76983861923219, 21.262298274040237, 25.627872842550296, 31.08484105318787, 37.906051316484835, 46.43256414560604, 57.09070518200754, 70.41338147750942, 87.06672684688677]


In [30]:
import dendropy

nexus_tree_path = '/Users/zhanghanni/Desktop/ASR_JoannesTree_Jun2023/J_newToL_BSrmd.nexus'

tree = dendropy.Tree.get(path=nexus_tree_path, schema='nexus')

# Get the internal nodes, excluding the root
internal_nodes = [node for node in tree.internal_nodes() if node.parent_node is not None]

commands = []

for idx, node in enumerate(internal_nodes):
    node_name = f"Node{idx}"
    node_taxa = node.leaf_nodes()
    
    taxa_labels = [taxon.taxon.label for taxon in node_taxa]
    
    # if None in taxa_labels:
    #     continue

    tag_command = f"AddTag {node_name} {' '.join(taxa_labels)}"
    commands.append(tag_command)
    
    mrca_command = f"AddMRCA {node_name} {node_name}"
    commands.append(mrca_command)

In [33]:
# 2.  make command files for BayesTraits
#naming: 'KL+{prior}' for KL diverence tests
count = 1
for p in hyperpriors:
    prior = p
    log = f'KL-{prior}' 
    prior = f'RevJumpHP exp 0 {prior}'

    output = open(f'KL_ToL_MRCACommands_{count}', 'w')
    output.write(f"1\n2\ncores 2\n{prior}\nIt -1\nLogFile {log}\n")
    count += 1
    # #run genome-same-phyla.py first
    # with open('/Users/zhanghanni/Desktop/ASR_JoannesTree_Jun2023/phyla_majorClade.txt', 'r') as file:
    #     contents = file.read()
    #     output.write(contents)

    for command in commands:
        output.write(command+"\n")
        #print(command)
    output.write("Run")
    output.close()